In [23]:
import pandas as pd 

df = pd.read_excel(r'C:\Users\Vusal\Downloads\new_data (2).xlsx')

In [24]:
df.head()

,Qiymət,C1,C2,Kateqoriya,Mərtəbə,Sahə,Otaq sayı,Kupça,İpoteka,Təmir
0,94500,40402511534715,498518545575805,0,5/5,53.0,2,1,0,1
1,126500,40404505013218,498509533353515,0,4/5,72.0,3,1,0,1
2,285000,404065973528723,49873263732787,1,4/18,122.2,3,1,0,1
3,87000,403766022856081,498478599128418,0,2/3,40.0,2,1,0,0
4,75000,404028671018614,49870570791354,0,3/5,30.0,2,1,0,1


In [25]:
df.columns = ['Price', 'C1', 'C2', 'Category', 'Floor', 'Area', 'RoomNumber', 'Kupcha', 'Mortgage', 'Remont']

In [26]:
df.head()

,Price,C1,C2,Category,Floor,Area,RoomNumber,Kupcha,Mortgage,Remont
0,94500,40402511534715,498518545575805,0,5/5,53.0,2,1,0,1
1,126500,40404505013218,498509533353515,0,4/5,72.0,3,1,0,1
2,285000,404065973528723,49873263732787,1,4/18,122.2,3,1,0,1
3,87000,403766022856081,498478599128418,0,2/3,40.0,2,1,0,0
4,75000,404028671018614,49870570791354,0,3/5,30.0,2,1,0,1


In [27]:
df.isna().sum()

Price         0
C1            0
C2            0
Category      0
Floor         0
Area          0
RoomNumber    0
Kupcha        0
Mortgage      0
Remont        0
dtype: int64

In [28]:
df['Floor_Home' ] = df.Floor.apply(lambda x: int(x.split('/')[0]))
df['Floor_House'] = df.Floor.apply(lambda x: int(x.split('/')[1]))

In [29]:
df.drop('Floor', axis = 1, inplace = True)

In [30]:
df.reset_index(inplace = True)

In [31]:
df

,index,Price,C1,C2,Category,Area,RoomNumber,Kupcha,Mortgage,Remont,Floor_Home,Floor_House
0,0,94500,40402511534715,498518545575805,0,53.0,2,1,0,1,5,5
1,1,126500,40404505013218,498509533353515,0,72.0,3,1,0,1,4,5
2,2,285000,404065973528723,49873263732787,1,122.2,3,1,0,1,4,18
3,3,87000,403766022856081,498478599128418,0,40.0,2,1,0,0,2,3
4,4,75000,404028671018614,49870570791354,0,30.0,2,1,0,1,3,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1927,1927,100000,404061226286124,498752005048461,0,55.0,2,1,0,1,7,9
1928,1928,175000,40398870782384,498763675142693,1,101.0,3,1,1,1,16,16
1929,1929,83000,403977447959129,498580169677734,0,41.0,2,1,0,1,1,9
1930,1930,140000,403978480607678,498743377190412,1,70.0,2,1,0,1,7,8


In [38]:
def extractCoordinates(col):
    col = str(col)
    col = col[:2] + '.' + col[2:]
    return float(col)

df['C1'] = df['C1'].apply(extractCoordinates)
df['C2'] = df['C2'].apply(extractCoordinates)

ValueError: could not convert string to float: '40..402511534715'

In [39]:
df.dtypes

index            int64
Price            int64
C1             float64
C2             float64
Category         int64
Area           float64
RoomNumber       int64
Kupcha           int64
Mortgage         int64
Remont           int64
Floor_Home       int64
Floor_House      int64
dtype: object

In [41]:
import geopy.distance

def calculateDistance(c1, c2):
    return geopy.distance.geodesic(c1, c2).km

In [42]:
calculateDistance((df.C1[0], df.C2[0]), (df.C1[5], df.C2[5]))

0.2921877535936418

In [43]:
metroCoordinates = [
    (40.3798499,49.8485742), #28May
    (40.4004909,49.8506117), #Ganjlik
    (40.4028298,49.8684909)] #Narimanov


In [44]:
diction = {}
index = 0
for i, j in zip(df['C1'], df['C2']):
    house_coordinates = tuple((i, j))
    distances = []
    for m in metroCoordinates:
        distances.append(calculateDistance(m, house_coordinates))
    diction[index] = (min(distances) * 1000)
    index+=1
    

In [45]:
distances = pd.DataFrame(diction, index = range(len(df))).T

distances.reset_index(inplace = True)
distances = distances[['index', 0]]
distances.rename({0:'DistanceToMetro'}, axis = 1, inplace = True)

In [46]:
distances

,index,DistanceToMetro
0,0,247.944726
1,1,446.679161
2,2,582.386062
3,3,365.687164
4,4,176.610399
...,...,...
1927,1927,676.829794
1928,1928,800.235805
1929,1929,698.718250
1930,1930,743.222874


In [47]:
df.merge(distances, on = 'index', how = 'inner')

,index,Price,C1,C2,Category,Area,RoomNumber,Kupcha,Mortgage,Remont,Floor_Home,Floor_House,DistanceToMetro
0,0,94500,40.402512,49.851855,0,53.0,2,1,0,1,5,5,247.944726
1,1,126500,40.404505,49.850953,0,72.0,3,1,0,1,4,5,446.679161
2,2,285000,40.406597,49.873264,1,122.2,3,1,0,1,4,18,582.386062
3,3,87000,40.376602,49.847860,0,40.0,2,1,0,0,2,3,365.687164
4,4,75000,40.402867,49.870571,0,30.0,2,1,0,1,3,5,176.610399
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1927,1927,100000,40.406123,49.875201,0,55.0,2,1,0,1,7,9,676.829794
1928,1928,175000,40.398871,49.876368,1,101.0,3,1,1,1,16,16,800.235805
1929,1929,83000,40.397745,49.858017,0,41.0,2,1,0,1,1,9,698.718250
1930,1930,140000,40.397848,49.874338,1,70.0,2,1,0,1,7,8,743.222874


In [48]:
df.drop('index', axis = 1, inplace = True)

In [49]:
df.head()

,Price,C1,C2,Category,Area,RoomNumber,Kupcha,Mortgage,Remont,Floor_Home,Floor_House
0,94500,40.402512,49.851855,0,53.0,2,1,0,1,5,5
1,126500,40.404505,49.850953,0,72.0,3,1,0,1,4,5
2,285000,40.406597,49.873264,1,122.2,3,1,0,1,4,18
3,87000,40.376602,49.847860,0,40.0,2,1,0,0,2,3
4,75000,40.402867,49.870571,0,30.0,2,1,0,1,3,5


In [51]:
from sklearn.tree import DecisionTreeClassifier

tree = DecisionTreeClassifier(criterion='entropy')

X = df.drop('Price', axis = 1)
y = df['Price']

tree.fit(X, y)

DecisionTreeClassifier(criterion='entropy')

In [53]:
prediction = tree.predict(X)

In [54]:
X

,C1,C2,Category,Area,RoomNumber,Kupcha,Mortgage,Remont,Floor_Home,Floor_House
0,40.402512,49.851855,0,53.0,2,1,0,1,5,5
1,40.404505,49.850953,0,72.0,3,1,0,1,4,5
2,40.406597,49.873264,1,122.2,3,1,0,1,4,18
3,40.376602,49.847860,0,40.0,2,1,0,0,2,3
4,40.402867,49.870571,0,30.0,2,1,0,1,3,5
...,...,...,...,...,...,...,...,...,...,...
1927,40.406123,49.875201,0,55.0,2,1,0,1,7,9
1928,40.398871,49.876368,1,101.0,3,1,1,1,16,16
1929,40.397745,49.858017,0,41.0,2,1,0,1,1,9
1930,40.397848,49.874338,1,70.0,2,1,0,1,7,8


In [56]:
from sklearn.metrics import accuracy_score

accuracy_score(y, prediction)

0.9994824016563147

In [73]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import *
from sklearn import tree
def printmetrics(y, prediction):
    print("Accuracy is " , accuracy_score (y, prediction))
    print(classification_report (y, prediction))
    
    
printmetrics(y, prediction)

Accuracy is  0.9994824016563147
              precision    recall  f1-score   support

       28000       1.00      1.00      1.00         1
       31000       1.00      1.00      1.00         1
       35000       1.00      1.00      1.00         1
       43000       1.00      1.00      1.00         1
       48000       1.00      1.00      1.00         1
       49000       1.00      1.00      1.00         1
       50000       1.00      1.00      1.00         1
       55000       1.00      1.00      1.00         4
       56000       1.00      1.00      1.00         2
       57000       1.00      1.00      1.00         3
       59000       1.00      1.00      1.00         1
       59800       1.00      1.00      1.00         1
       60000       1.00      1.00      1.00         3
       61600       1.00      1.00      1.00         1
       62000       1.00      1.00      1.00         1
       63000       1.00      1.00      1.00         2
       63500       1.00      1.00      1.00      